In [ ]:
# imports
import pickle
import numpy as np
import pandas as pd
from sklearn import preprocessing # pip install sklearn doesnt work

#pip install tensorflow-gpu
#export CUDA_VISIBLE_DEVICES=0,1
import tensorflow as tf
tf.config.list_physical_devices()


In [ ]:
samples = 20

In [ ]:
train_df = pd.read_csv('train.csv' , index_col= 0)
train_df.drop(['date','user'], axis = 1,inplace = True)
train_df.shape

In [ ]:
train_df.head()

In [ ]:
target = train_df['activity']
train_df.drop('activity', axis = 1 , inplace=True)

In [ ]:
target_sampled = target[::samples]
type(target_sampled) , target_sampled.shape

In [ ]:
y_train = np.array(target_sampled)

In [ ]:
X_train = train_df.values.reshape(int(train_df.shape[0]/samples)
                                  ,samples*train_df.shape[1]
                                  ,1)

In [ ]:
f' X_train shape : {np.shape(X_train) }  ||  X_train[0] shape : {np.shape(X_train[0])}'

In [ ]:
type(X_train) , type(y_train)

In [ ]:
np.unique(y_train)

In [ ]:
len(X_train) , len(y_train)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train , y_train, test_size=0.05, random_state=42 )

### Label encoding

In [ ]:
le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)
list(le.classes_) , y_train

In [ ]:
with open('encoder.pkl','wb') as f:
    pickle.dump(le, f)
    
with open('encoder.pkl', 'rb') as f:
    le = pickle.load(f) 

In [ ]:
np.shape(X_train),np.shape(y_train) , np.shape(X_test),np.shape(y_test)

# Modeling

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Input, Flatten
from keras.activations import relu
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

In [ ]:
model = Sequential()
model.add(Input(shape=(80,1))) # Input tensor
model.add(LSTM(units = 512, return_sequences = True ))
model.add(Dropout(0.2))
model.add(LSTM(units = 128, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 64, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 32, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 16))
model.add(Dropout(0.2))
model.add(Dense(units = 6, activation = "softmax"))

In [ ]:
#model.add(Flatten())

model.compile(
    optimizer = Adam(learning_rate=0.002),
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
             )

In [ ]:
early_stopping = EarlyStopping(
    patience = 3,
    monitor='accuracy',
    restore_best_weights = True
    )


In [ ]:
history = model.fit(
    X_train,y_train,
    epochs = 32,
    verbose = 2,
    batch_size = 20,
    callbacks=[early_stopping,],
    )

In [ ]:
X = X_train[0]
X = X.reshape(1,80,1)
np.shape(X)

In [ ]:
Y = X.tolist()
np.shape(Y)

In [ ]:
list(le.inverse_transform([5])) # must be list type or np array

In [ ]:
preds = model.predict(Y).argmax()
list(le.inverse_transform([preds]))

In [ ]:
model.save('model')

In [ ]:
np.shape(X_test[0])

In [ ]:
list(X_test[0].reshape(80))

In [ ]:
MODEL = tf.keras.models.load_model('model/')

preds = []
preds.append(MODEL.predict(X_test).tolist())
#list(le.inverse_transform([preds]))
preds

In [ ]:
y_preds = [ np.argmax(x) for x in preds[0] ]

In [ ]:
preds = le.inverse_transform(y_preds)

In [ ]:
np.shape(preds) ,np.shape(X_test)

In [ ]:
pred_df = pd.DataFrame()
pred_df['preds'] = preds

In [ ]:
pred_df['y_test'] = y_test
pred_df

In [ ]:
np.shape(preds)

In [ ]:
pred_df[pred_df['y_test'] == pred_df['preds']]

# Transforming test set

In [ ]:
np.shape(X_test)

In [ ]:
np.save('test', X_test)

In [ ]:
X_test = np.load('test.npy')
np.shape(X_test)

In [ ]:
import numpy as np
x = np.load('test.npy')
x